## Install dependencies

In [9]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

## Load data

In [10]:
LSA = pd.read_csv('LSA.csv')
bussgeld = pd.read_csv('bussgeld.csv')

### Look at the first rows

In [11]:
LSA.head()

,LSA-Nr.,Standort,X,Y,zipcode
0,1617,Aachener Straße / Aachener Weiher / Richard-Wa...,6.928991,50.936434,50674
1,3634,Aachener Straße / Alter Militärring / Auf dem ...,6.881020,50.937725,50933
2,3656,Aachener Straße / Am Rapohl / Bodelschwinghstraße,6.822827,50.938514,50859
3,3639,Aachener Straße / Am Römerhof,6.867658,50.937803,50933
4,3638,Aachener Straße / Am Sportpark Müngersdorf / M...,6.870817,50.937833,50933


In [12]:
bussgeld.head()

,Unnamed: 0,Jahr,Monat,datum_von,kennzeichen1,fahrzeugart,fabrikat,strasse,hausnummer,tatbestand1,tatbestand2,tatbestand3,latAndlong
0,0,2021,1,2021-01-14 08:32:00.000,MG,PKW,FIAT,BEETHOVENSTR.,Mai 13,141324,NaN,NaN,"(50.9316361, 6.9370651)"
1,1,2021,1,2021-01-14 13:23:00.000,BM,PKW,OPEL,HOHENSTAUFENRING,61,141322,NaN,NaN,"(50.9343146, 6.9391205)"
2,2,2021,1,2021-01-14 13:23:00.000,K,PKW,VW-VOLKSWAGEN,HOHENSTAUFENRING,57A,141322,NaN,NaN,"(50.9340377, 6.9392313122363465)"
3,3,2021,1,2021-01-14 09:25:00.000,K,PKW,FORD,LINDENSTR.,52,141324,NaN,NaN,"(50.9336412, 6.9353042)"
4,4,2021,1,2021-01-14 14:25:00.000,M,PKW,VW-VOLKSWAGEN,MOLTKESTR.,NaN,141322,NaN,NaN,"(50.9363573, 6.9327157)"


In [13]:
bussgeld = bussgeld[['datum_von', 'kennzeichen1', 'fahrzeugart', 'fabrikat', 'latAndlong']]
bussgeld['latAndlong'] = bussgeld['latAndlong'].progress_apply(lambda x : eval(x))
bussgeld[['Y','X']] = bussgeld['latAndlong'].progress_apply(pd.Series)
bussgeld = bussgeld[bussgeld['X'] != 0.0]
bussgeld.drop('latAndlong', axis=1)
bussgeld.head()


  0%|          | 0/147018 [00:00<?, ?it/s]

  0%|          | 0/147018 [00:00<?, ?it/s]

,datum_von,kennzeichen1,fahrzeugart,fabrikat,latAndlong,Y,X
0,2021-01-14 08:32:00.000,MG,PKW,FIAT,"(50.9316361, 6.9370651)",50.931636,6.937065
1,2021-01-14 13:23:00.000,BM,PKW,OPEL,"(50.9343146, 6.9391205)",50.934315,6.939120
2,2021-01-14 13:23:00.000,K,PKW,VW-VOLKSWAGEN,"(50.9340377, 6.9392313122363465)",50.934038,6.939231
3,2021-01-14 09:25:00.000,K,PKW,FORD,"(50.9336412, 6.9353042)",50.933641,6.935304
4,2021-01-14 14:25:00.000,M,PKW,VW-VOLKSWAGEN,"(50.9363573, 6.9327157)",50.936357,6.932716


In [14]:
from scipy.spatial import distance
def get_nearest_trafficlight(X, Y, dataTrafficLight):
    # Define the violation location coordinates (latitude and longitude)
    violation_X= X 
    violation_Y = Y 
    
    # Calculate the distance between the violation location and each traffic light
    distances = []
    for index, row in dataTrafficLight.iterrows():
        light_X = row['X']  
        light_Y = row['Y']  

        # Calculate the Euclidean distance between the violation location and the current traffic light
        dist = distance.euclidean((violation_X, violation_Y), (light_X, light_Y))
        distances.append(dist)

    # Find the index of the nearest traffic light
    nearest_light_index = distances.index(min(distances))
    return dataTrafficLight.iloc[nearest_light_index]['LSA-Nr.']

In [15]:
bussgeld['LSA_Nr'] = bussgeld[['X','Y']].progress_apply(lambda r: get_nearest_trafficlight(r['X'],r['Y'],LSA), axis=1)

  0%|          | 0/142491 [00:00<?, ?it/s]

We rename the column names here before merging both the datasets

In [16]:
LSA.rename(columns={"LSA-Nr.": "LSA_Nr", 'X': "LSA_X", "Y": "LSA_Y"},inplace= True)

In [17]:
LSA.head()

,LSA_Nr,Standort,LSA_X,LSA_Y,zipcode
0,1617,Aachener Straße / Aachener Weiher / Richard-Wa...,6.928991,50.936434,50674
1,3634,Aachener Straße / Alter Militärring / Auf dem ...,6.881020,50.937725,50933
2,3656,Aachener Straße / Am Rapohl / Bodelschwinghstraße,6.822827,50.938514,50859
3,3639,Aachener Straße / Am Römerhof,6.867658,50.937803,50933
4,3638,Aachener Straße / Am Sportpark Müngersdorf / M...,6.870817,50.937833,50933


In [18]:
merged_data = pd.merge(bussgeld, LSA, on="LSA_Nr")

In [19]:
merged_data.head()

,datum_von,kennzeichen1,fahrzeugart,fabrikat,latAndlong,Y,X,LSA_Nr,Standort,LSA_X,LSA_Y,zipcode
0,2021-01-14 08:32:00.000,MG,PKW,FIAT,"(50.9316361, 6.9370651)",50.931636,6.937065,1458,Roonstraße / Beethovenstraße / Rathenauplatz,6.936945,50.931568,50674
1,2021-01-14 08:32:00.000,LU 7,PKW,OPEL,"(50.9316361, 6.9370651)",50.931636,6.937065,1458,Roonstraße / Beethovenstraße / Rathenauplatz,6.936945,50.931568,50674
2,2021-01-14 10:25:00.000,K,PKW,DACIA,"(50.9316361, 6.9370651)",50.931636,6.937065,1458,Roonstraße / Beethovenstraße / Rathenauplatz,6.936945,50.931568,50674
3,2021-01-14 10:25:00.000,K,PKW,FORD,"(50.9316361, 6.9370651)",50.931636,6.937065,1458,Roonstraße / Beethovenstraße / Rathenauplatz,6.936945,50.931568,50674
4,2021-01-14 08:32:00.000,MG,PKW,FIAT,"(50.9316361, 6.9370651)",50.931636,6.937065,1458,Roonstraße / Beethovenstraße / Rathenauplatz,6.936945,50.931568,50674


In [20]:
merged_data.to_csv("finalData.csv", index=False)